# SimpleBGC controller

Implemented using the serial API specified in https://www.basecamelectronics.com/serialapi/ and by looking at the source code for the arduino library, found at https://github.com/alexmos/sbgc-api-examples. It is very incomplete (I only added the commands I need) but should be easy enough to follow that other commands can be added without much extra work.

Main commands (prefixed with `gimbal_control.`):
1. `turn_on_motors()`
2. `send_angle_command(roll=, pitch=, yaw=)`
3. `angles_dict = get_motor_angles()`
4. `turn_off_motors()`

All angles are in degrees

## Setup

In [ ]:
import numpy as np, serial, time

In [ ]:
ser = serial.Serial('/dev/serial0',  # pin 8 = Tx, pin 10 = Rx, has been mapped to ttyAMA0
                    baudrate=115200,
                    bytesize=serial.EIGHTBITS,
                    parity=serial.PARITY_NONE,
                    stopbits=serial.STOPBITS_ONE,
                    timeout=0.05)  # 50 ms timeout

#serial0 == '/dev/ttyS0', connected on physical pins 8 & 10.
# parity issues on ttyS0:
#     https://github.com/pyserial/pyserial/issues/196#issuecomment-323382976

# therefore, these pins have been mapped to ttyAMA0 which has better support
#serial1 == '/dev/ttyAMA0', connected internally to the Bluetooth hardware.

In [ ]:
# incomplete lookup table - see the note in last cell
L = {
    # menu items
    'CMD_EXECUTE_MENU': 69,
    'SBGC_MENU_MOTOR_ON': 11,
    'SBGC_MENU_MOTOR_OFF': 12,
    
    # cmd items
    'CMD_CONTROL': 67,
    'CMD_REALTIME_DATA_CUSTOM': 88
}

## Utils

In [ ]:
def as_int(b):
    """ Converts b (a bytearray) into a signed integer.
        Assumes little endian """
    return int.from_bytes(b, byteorder='little', signed=True)

def as_uint(b):
    """ Converts b (a bytearray) into an unsigned integer.
        Assumes little endian """
    return int.from_bytes(b, byteorder='little', signed=False)

def print_binary_padded(hex_data, num_bits=8, reverse=False):
    """ Prints the hex value in binary, with zero padding
    
        >>> print_binary_padded(0x03) # --> 0b00000011
        >>> print_binary_padded(0x0421, num_bits=16) # --> 0b0000010000100001
    """
    if reverse is True:
        print('LSB first: 0b' + bin(hex_data)[2:].zfill(num_bits)[::-1])
    else:
        print('0b' + bin(hex_data)[2:].zfill(num_bits))  # [2:] to get rid of the '0b' part

def print_hex_nicely(hex_string):
    """ Prints a hex string in groups of bytes
        Useful when you want to decode a bytearray by eye
    
        >>> print('Response:   ', end='')
        >>> print_hex_nicely(b'>X\x08`\x0c\xf3z\xffr\xfc\x8e\x03w'.hex())
    """
    [print(hex_string[i*2:i*2+2] + ' ', end='') for i in range(len(hex_string)//2)]
    print()

def print_twos_complement(number, num_bits=16):
    """ Another function made while debugging.
    >>> print_twos_complement(-4, num_bits=8)  # --> 11111100
    """
    print(format(number % (1 << num_bits), '0' + str(num_bits) + 'b'))

## Main command functions

In [ ]:
def send_data(command_ID, data=None, data_size=0, debug=False):
    """ Compose and send a command to the SimpleBGC
    Inputs:
        command_ID:     a string, such as 'CMD_GET_ANGLES'
        data:           a list of numpy scalars
        data_size:      the number of bytes in 'data'

    Format:
        head:
            start_char = ord('>') = 0x3E. 1u
            command_ID. 1u
            data_size. 1u. Can be 0
            header_checksum = (command ID + data_size) % 255. 1u

        body:
            array_of_bytes_of_length_data_size
            body_checksum. 1u
    """
    # compose head:
    start_char = np.uint8(ord('>'))
    header_checksum = np.uint8(L[command_ID] + data_size)
    
    message = bytearray()
    message.append(start_char)
    message.append(L[command_ID])
    message.append(np.uint8(data_size))
    message.append(header_checksum)

    # compose body:
    body_checksum = 0
    if data_size > 0:
        for d in data:
            if d.nbytes == 1:
                message.append(d)
                body_checksum += d
            elif d.nbytes == 2:
                d_bytes = d.tobytes()
                message.append(d_bytes[0])  # working with little endian
                message.append(d_bytes[1])
                body_checksum += d_bytes[0] + d_bytes[1]
            else:
                print('gimbal_control: Haven\'t yet built in functionality for 3 or more bytes')
        
        if debug: print('gimbal_control: body_checksum = %i' % body_checksum)

        message.append(np.uint8(body_checksum))
    
    ser.flushInput()
    ser.flushOutput()
    ser.flushInput()
    ser.flushOutput()
    ser.write(message)
    
    if debug:
        print('gimbal_control: message sent:\t\t', end='')
        print_hex_nicely(message.hex())
    
    return message

In [ ]:
def send_angle_command(roll, pitch, yaw, debug=False):
    """ send an angle command to the gimbal
        roll, pitch and yaw are in degrees """
    
    # the spec sheet says to send CONTROL_MODE three times (one for each axis)
    # BUT it doesn't seem to work when I do that. Not sure why

    scaling = 0.02197265625  # units/degree. 2**15 * 0.02197265625 == 720
    message = [
        np.uint8(2),                # CONTROL_MODE = MODE_ANGLE
        np.int16(0),                # roll speed
        np.int16(roll/scaling),     # roll angle
      - np.int16(0),                # pitch speed (now pitch up = positive)
      - np.int16(pitch/scaling),    # pitch angle (now pitch up = positive)
        np.int16(0),                # yaw speed
        np.int16(yaw/scaling)       # yaw angle
    ]

    return send_data('CMD_CONTROL', data=message, data_size=13, debug=debug)

In [ ]:
def send_speed_command(roll, pitch, yaw, debug=False):
    """ send a speed command to the gimbal
        roll, pitch and yaw are in degrees/second """
    
    # the spec sheet says to send CONTROL_MODE three times (one for each axis)
    # BUT it doesn't seem to work when I do that. Not sure why
    
    scaling = 0.1220740379          # units/degree/s
    message = [
        np.uint8(1),                # CONTROL_MODE = MODE_SPEED
        np.int16(roll/scaling),     # roll speed
        np.int16(0),                # roll angle
      - np.int16(pitch/scaling),    # pitch speed
      - np.int16(0),                # pitch angle
        np.int16(yaw/scaling),      # yaw speed
        np.int16(0)                 # yaw angle
    ]

    return send_data('CMD_CONTROL', data=message, data_size=13, debug=debug)

In [ ]:
def get_motor_angles(debug=False):
    """ Get the gimbal's angles, as measured by the IMU and estimated by the SBGC EKF
        Units are in degrees.
        
        One could extend this to fetch other useful data as well - simple bit shift
        and then correct extraction of the data. See page 47 of the serial api doc.
    """

    msg = [np.uint8(1),  # 1 = activated bit 0
           np.uint8(0), np.uint8(0), np.uint8(0), np.uint8(0), np.uint8(0),
           np.uint8(0), np.uint8(0), np.uint8(0), np.uint8(0)]#np.uint64(0)]  # empty flag stuff/reserved
    send_data('CMD_REALTIME_DATA_CUSTOM', msg, 10, debug)
    
    SBGC_response = ser.read(13)  # read (up to) 13 bytes

    if response_is_not_valid(SBGC_response):
        return 0
    
    # example responses:
    # 3e 58 08 60   f6 89   1a ff   fe ff   fe ff   92   # 0 deg angles
    # 3e 58 08 60   66 a5   76 ff   74 fc   90 03   83   # 20 deg angles

    # they each arrive in 14-bit resolution and require scaling
    scaling = 0.02197265625  # scales to degrees
    angles = {
        'timestamp_ms': as_uint(SBGC_response[4:6]),
        'roll':         as_int(SBGC_response[6:8]) * scaling,
        'pitch':       -as_int(SBGC_response[8:10]) * scaling, # change from up = neg to up = pos
        'yaw':          as_int(SBGC_response[10:12]) * scaling
    }
    
    return angles

In [ ]:
def turn_off_motors(debug=False):
    message = [np.uint8(L['SBGC_MENU_MOTOR_OFF'])]
    return send_data('CMD_EXECUTE_MENU', message, 1, debug)

def turn_on_motors(debug=False):
    message = [np.uint8(L['SBGC_MENU_MOTOR_ON'])]
    return send_data('CMD_EXECUTE_MENU', message, 1, debug)

In [ ]:
def response_is_not_valid(SBGC_response):
    """ should be used as,
        if response_is_not_valid(SBGC_response):
            return -1
    """
    if SBGC_response == b'':
        print('gimbal_control: No response response from controller')
        return True
    
    elif chr(SBGC_response[0]) != '>':
        print('gimbal_control: Invalid start of response. Got:', SBGC_response)
        return True
    
    elif (SBGC_response[1] + SBGC_response[2] != SBGC_response[3]):
        print('gimbal_control: Invalid header checksum in response. Got:', SBGC_response)
        return True
    
    # maybe look at the checksum for the body? though not knowing the datatype would be an issue
    # elif ...
    
    else:
        return False

## Debugging

In [ ]:
if __name__ == '__main__':
    debug = True
    turn_off_motors(debug)
    time.sleep(3)
    turn_off_motors(debug)
    time.sleep(3)

    roll = 0
    for pitch in [45, 30, 15, 0, -10]:
        for yaw in [-60, -30, 0, 30, 60]:
            send_angle_command(roll, pitch, yaw, debug)
            time.sleep(2)
            angles = get_motor_angles(debug)
            print('Roll = %d, pitch = %d, yaw = %d' % (angles['roll'], angles['pitch'], angles['yaw']))

## Complete lookup table

Check the SerialAPI document for the rest of the commands. I specifically didn't include a lookup table (ie extend `L` above) with all commands as implementing a command isn't always as simple as sending it. You usually need a special receive syntax, etc, so it's a bit misleading to show the whole table. Add stuff to the other `L` thing if needed